# Atomic multiplets

Two different atomic shells are considered.

Each shell is characterized by $(n,l)$, where $n$ is the principal quantum number and $l$ is the angular momentum.

One shell is initially considered to be fully occupied.
Different occupations is considered for the second shell. 

The Hund's ground state is considered as the initial configuration.
Then an electron is moved from the fully occupied shell to the other shell.
All multiplets in the excited configuration are calculated. 
Using dipole selection rules the allowed transitions from the Hund's ground state to the multiplets are obtained.  


In [80]:
import itertools


def hunds_ground_state(nb, n):
    '''
    Return L, S and J for the ground state according to Hunds rule.
    
    Parameters
    ----------
    nb : int
        Number of spin-orbitals
    n : int
        Number of electrons
    '''
    assert 0 <= n <= nb
    assert nb % 2 == 0
    l = (nb-2)//4
    S, L = 0, 0
    counter = 0
    for sz in [1/2, -1/2]:
        for lz in range(l,-l-1,-1):
            if counter < n:
                S += sz
                L += lz
                counter += 1
            if counter == n:
                J = L+S if nb/2 < n else abs(L-S) 
                return (L, S, J)            


def find_multiplets(nb1,n1,nb2,n2):
    '''
    Return all multiplets given the occupation in two shells.
    
    Parameters
    ----------
    nb1 : int
        Number of spin-orbitals in the first shell
    n1 : int
        Number of electrons in the first shell
    nb2 : int
        Number of spin-orbitals in the second shell
    n2 : int
        Number of electrons in the second shell
        
    '''
    # Angular momentum
    l1 = (nb1-2)//4
    l2 = (nb2-2)//4
    # Check validity of input data
    if not (nb1%2 == 0 and (nb1-2)% 4 == 0): 
        raise ValueError('nb1 needs to fulfill: nb1 = 2*(2*L+1), L non-negative integer') 
    if not (nb2%2 == 0 and (nb2 -2)%4 == 0): 
        raise ValueError('nb2 needs to fulfill: nb2 = 2*(2*L+1), L non-negative integer') 

    # Create basis.
    # For each shell, create all configurations
    # given the occupation in that shell.
    spin_orbitals1 = [(l1, lz, sz) for lz in range(-l1, l1+1) for sz in (-1/2, 1/2)]
    spin_orbitals2 = [(l2, lz, sz) for lz in range(-l2, l2+1) for sz in (-1/2, 1/2)]
    states1 = tuple(itertools.combinations(spin_orbitals1, n1))
    states2 = tuple(itertools.combinations(spin_orbitals2, n2))
    states = []
    for s1 in states1:
        for s2 in states2:
            states.append(s1 + s2)
    states = tuple(states)
    #print 'Number of states:',len(states)
    
    # Measure Lz and Sz for each state
    n_LzSz_configurations = {}
    for state in states:
        Lz = 0
        Sz = 0
        # Loop over occupied spin-orbitals in one configuration
        for spin_orbital in state:
            Lz += spin_orbital[1]
            Sz += spin_orbital[2]
        if (Lz, Sz) in n_LzSz_configurations:
            n_LzSz_configurations[(Lz, Sz)] += 1
        else:
            n_LzSz_configurations[(Lz, Sz)] = 1
    
    # Calculate L and S
    LS = []
    while n_LzSz_configurations:
        # Find Lz max
        Lz_max = max([Lz for (Lz, Sz) in n_LzSz_configurations])         
        # find Sz max, given Lz max 
        Sz_max = max([Sz for (Lz, Sz) in n_LzSz_configurations if Lz == Lz_max])
                
        # Save found L and S
        LS.append((Lz_max, Sz_max))
        # Remove one configuration from each (Lz,Sz) pair fullfilling
        # |Lz| <= Lz_max and |Sz| <= Sz_max
        for (Lz, Sz) in tuple(n_LzSz_configurations):
            if abs(Lz) <= Lz_max and abs(Sz) <= Sz_max:
                if n_LzSz_configurations[(Lz, Sz)] == 1:
                    n_LzSz_configurations.pop((Lz, Sz))    
                else:
                    n_LzSz_configurations[(Lz, Sz)] -= 1
                        
    multiplets = []
    for (L, S) in LS:
        for J in np.arange(abs(L-S), L+S+1):
            multiplets.append((L, S, J))
    return multiplets


In [87]:
angular_momentum_symbols = {0:"s", 1:"p", 2:"d", 3: "f"}

# These variable are unsed except for printing
principal_quantum_number1 = 2
principal_quantum_number2 = 3

# Select angular momentum for the two shells
l1, l2 = 1, 2

# Number of spin-orbitals for the two shells
nb1, nb2 = 4*l1 + 2, 4*l2 + 2 
# Consider the first shell to be fully occupied
n1 = nb1

print('         Transition        GroundState(L,S,J)  #Transitions #TermSymbols')
# Loop over all possible occupations in the second shell
for n2 in range(nb2):    
    LSJ0 = hunds_ground_state(nb2, n2)
    # Consider situation where one electron has moved from the first shell to the second shell
    multiplets = find_multiplets(nb1, n1-1, nb2, n2+1)
    transitions = []
    for m in multiplets:
        # Dipole selection rules: DeltaJ = -1,1 or DeltaJ=0 if J>0
        if abs(m[2]-LSJ0[2])==1 or (m[2]==LSJ0[2] and m[2]>0):
            transitions.append(m)
    s_initial = f"({principal_quantum_number1}{angular_momentum_symbols[l1]}^{n1} {principal_quantum_number2}{angular_momentum_symbols[l2]}^{n2})" 
    s_excited = f"({principal_quantum_number1}{angular_momentum_symbols[l1]}^{n1-1} {principal_quantum_number2}{angular_momentum_symbols[l2]}^{n2+1})" 
    s_transition = f"{f'{s_initial} -> {s_excited}':30}"
    print(s_transition + f"{f'{LSJ0}':15}  {len(transitions):8} {len(multiplets):8}")


         Transition        GroundState(L,S,J)  #Transitions #TermSymbols
(2p^6 3d^0) -> (2p^5 3d^1)    (0, 0, 0)               3       12
(2p^6 3d^1) -> (2p^5 3d^2)    (2, 0.5, 1.5)          29       45
(2p^6 3d^2) -> (2p^5 3d^3)    (3, 1.0, 2.0)          68      110
(2p^6 3d^3) -> (2p^5 3d^4)    (3, 1.5, 1.5)          95      180
(2p^6 3d^4) -> (2p^5 3d^5)    (2, 2.0, 0.0)          32      214
(2p^6 3d^5) -> (2p^5 3d^6)    (0, 2.5, 2.5)         110      180
(2p^6 3d^6) -> (2p^5 3d^7)    (2, 2.0, 4.0)          53      110
(2p^6 3d^7) -> (2p^5 3d^8)    (3, 1.5, 4.5)          16       45
(2p^6 3d^8) -> (2p^5 3d^9)    (3, 1.0, 4.0)           4       12
(2p^6 3d^9) -> (2p^5 3d^10)   (2, 0.5, 2.5)           1        2
